In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up Selenium
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Function to scrape Pinterest for a celebrity
def scrape_pinterest(celebrity_name):
    search_query = f"{celebrity_name} latest fashion"
    url = f"https://www.pinterest.com/search/pins/?q={search_query}"
    driver.get(url)
    
    # Wait until the pins are loaded
    try:
        pins = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[data-test-id="pin"]'))
        )
    except Exception as e:
        print(f"An error occurred: {e}")
        driver.quit()
        return []
    
    pin_urls = []

    for pin in pins[:7]:  # Get the latest 7 pins
        try:
            pin_url = pin.find_element(By.TAG_NAME, 'a').get_attribute('href')
            pin_urls.append(pin_url)
        except Exception as e:
            print(f"An error occurred while processing a pin: {e}")
    
    return pin_urls

# Example usage
celebrity = "Alia Bhatt Latest Dresses"
latest_pins = scrape_pinterest(celebrity)
print(latest_pins)

driver.quit()


['https://www.pinterest.com/pin/291185932149816058/', 'https://www.pinterest.com/pin/148970700167842011/', 'https://www.pinterest.com/pin/794181715529613812/', 'https://www.pinterest.com/pin/588634613793321472/', 'https://www.pinterest.com/pin/851039660843658428/', 'https://www.pinterest.com/pin/789115165968114235/', 'https://www.pinterest.com/pin/593067844659790690/']


In [29]:
import requests
from bs4 import BeautifulSoup
import os

# List of Pinterest links
pin_links = latest_pins 

# Create a directory to save images
os.makedirs('pinterest_images', exist_ok=True)

def download_image(url, path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded {path}")
    else:
        print(f"Failed to download {url}")

def get_image_url(pin_url):
    response = requests.get(pin_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # Find the meta tag with the property og:image
    meta_tag = soup.find('meta', property='og:image')
    if meta_tag:
        return meta_tag['content']
    return None

# Loop through each pin link to find and download images
for i, pin_link in enumerate(pin_links):
    image_url = get_image_url(pin_link)
    if image_url:
        download_image(image_url, f'pinterest_images/image_{i+1}.jpg')
    else:
        print(f"No image found for {pin_link}")

print("Download complete.")


Downloaded pinterest_images/image_1.jpg
Downloaded pinterest_images/image_2.jpg
Downloaded pinterest_images/image_3.jpg
Downloaded pinterest_images/image_4.jpg
Downloaded pinterest_images/image_5.jpg
Downloaded pinterest_images/image_6.jpg
Downloaded pinterest_images/image_7.jpg
Download complete.
